web_application.ipynb: 
    This notebook contains the code for building the web application for model deployment.

1. Building the web application interface.
2. Integrating the model with the web application.
3. Testing the web application.